# 优先级经验回放
***离线算法总是跟经验池离不开关系***

但是一般的随机抽样不能够将有用的经验反复学习，所以有了PER-Prioritized Experience Replay

1. 如果算法足够好，没必要用PER.
2. 如果奖励函数足够dense，也没有必要用PER.
3. 因为用了PER速度大大降低，性能不一定会提高。
4. 如果真实场景交互，CPU资源足够，稀疏奖励，可以试试。

## 回顾DQN中的ER-Experience Replay
1. 保持队列的结构，先进先出，当长度到达定值后开始踢人
2. 从整个Buffer中均匀分布随机选择
3. 然而，在DQN数以千计的样本，也就是 [s, a, r, s’] 中，一定有一些样本可以帮助DQN更快的收敛；或者说，存在一些样本，如果我们先学了这些样本，后面的就不用再学了。

In [1]:
import numpy as np
class SumTree():
    def __init__(self,max_mem) -> None:
        self.capacity=max_mem
        self.data_ptr=0
        # 对于capacity个数据节点，我们需要使用capacity-1个父节点来形成一个树形结构
        # 那么一共需要 2*capacity-1个节点，定义tree[:self.capacity-1]为父节点，后面的是叶子节点
        self.tree=np.zeros(2*self.capacity-1)
        #存储数据的节点，与tree[self.capacity-1:] 一一对应
        self.data=np.zeros(self.capacity,dtype=object)
    
    def add(self,priority:float,data:object):
        # 叶子节点的下标
        tree_idx = self.data_ptr+self.capacity-1
        # 存储数据
        self.data[self.data_ptr]=data
        # 更新父节点的权重
        self.update_parent_weights(tree_idx,priority)
        # 如果达到了存储的上限，则踢掉刚刚进来的数据
        self.data_ptr+=1
        if self.data_ptr>=self.capacity:
            self.data_ptr=0

    def update_parent_weights(self,tree_idx:int,priority:float):
        # 获得修改的值
        change_value = priority - self.tree[tree_idx]
        # 直接修改叶子节点的值
        self.tree[tree_idx]=priority
        # 跟这个叶子节点有关的父亲节点 全部要更新
        while tree_idx != 0:
            tree_idx=(tree_idx-1)//2
            self.tree[tree_idx]+=change_value

    def get_leaf(self,v:float):
        """
        根据采样到的值v来取叶子节点的数据
        """
        parent_idx=0
        while True:
            cl_idx=2*parent_idx+1
            cr_idx=cl_idx+1
            # 超出界限了就结束搜索
            if cl_idx>=len(self.tree):
                leaf_idx=parent_idx
                break
            else:
                if v<=self.tree[cl_idx]:
                    parent_idx=cl_idx
                else:
                    v=v-self.tree[cl_idx]
                    parent_idx=cr_idx
        data_idx=leaf_idx-(self.capacity-1)
        # return leaf_idx,and the priority of the idx, and the data
        return leaf_idx,self.tree[leaf_idx],self.data[data_idx]

    @property
    def total_priority(self):
        return self.tree[0]
        

# 定义PER类

In [2]:
class ProritizedExperienceReplay():
    def __init__(self,
    max_size:int,
    batch_size:int,
    clipped_abs_error:float,
    epsilon:float,
    alpha:float,
    beta:float,
    beta_increment:float,
    ) -> None:
        """
        Params
        ------
        alpha:trade-off factor 控制采样在uniform和greedy的偏好,0代表均匀随机，1代表完全按贪婪算法
        epsilon: 避免除数为0的一个很小的数
        beta: 决定抵消PER对于收敛结果的影响，1代表完全抵消，这样就与ER没有区别了
        beta_increment: 每次采样对beta采取的增量
        clipped_abs_error: abs_error的上限
        """

        #存储参数
        self.clipped_abs_error = clipped_abs_error
        self.batch_size = batch_size
        self.beta_increment=beta_increment
        self.epsilon=epsilon
        self.alpha=alpha
        self.beta=beta
        # 定义SUMTREE
        self.tree=SumTree(max_size)

    def store_transition(self,transition:object):
        # 索引后面的叶子节点
        max_priority=np.max(self.tree.tree[-self.tree.capacity:])
        if max_priority==0:
            max_priority=self.clipped_abs_error
            #存进来的时候，调成最大的优先级，保证每个样本都要过一遍
        self.tree.add(max_priority,transition)
    
    def sample(self):
        """
        Returns
        -------
        Tree index : the indexs of the tree
        Data : the data of the nodes
        ISWeights : the normalized Priority of the nodes
        """
        b_idx=np.empty((self.batch_size),dtype=np.int32)
        b_memory=np.empty((self.batch_size,self.tree.data[0].size))
        ISWeights = np.empty((self.batch_size, 1))
        #分成batch_size 个段
        priority_segments= self.tree.total_priority/self.batch_size
        # 递增的beta
        self.beta=np.min([1.,self.beta+self.beta_increment])
        # 取得最小倍率，这是normalization的过程
        min_prob = np.min(self.tree.tree[-self.tree.capacity])/self.tree.total_priority

        for i in range(self.batch_size):
            # 将优先值分为batch_size片，然后从每片中均匀采样
            a,b = priority_segments*i,priority_segments*(i+1)
            sample_val=np.random.uniform(a,b)
            idx,p,data = self.tree.get_leaf(sample_val)
            #取得优先值的占比
            prob=p/self.tree.total_priority
            ISWeights[i,0]=np.power(prob/min_prob,-self.beta)
            b_idx[i],b_memory[i,:]=idx,data
        # 返回抽样值
        return b_idx,b_memory,ISWeights

    def batch_update(self,tree_idx:np.ndarray,abs_errors:np.ndarray):
        # + epsilon避免除以0
        abs_errors+=self.epsilon
        clipped_errors=np.minimum(abs_errors,self.clipped_abs_error)
        ps = np.power(clipped_errors,self.alpha)
        for ti,p in zip(tree_idx,ps):
            self.tree.update_parent_weights(ti,p)


# TEST

In [4]:
mem = ProritizedExperienceReplay(5,1,100.0,1e-5,1,1,0.001)
for i in range(5):

    mem.store_transition((np.array([i+1.,i+2.])))
for i in range(5):
    mem.batch_update(np.array([4,5,6,7,8]),np.array([1,2,3,4,5],dtype=np.float32))

sum_sample = 100000
index_count = np.zeros((5))
for i in range(sum_sample):
    id,_,_=mem.sample()
    index_count[id-4]+=1

for i in range(5):
    print(
        f"id:{i}->propotion:{index_count[i]/sum(index_count)*100 :.2f} vs target weights {(i+1)/15 *100 :.2f}%")


id:0->propotion:6.71 vs target weights 6.67%
id:1->propotion:13.33 vs target weights 13.33%
id:2->propotion:19.96 vs target weights 20.00%
id:3->propotion:26.90 vs target weights 26.67%
id:4->propotion:33.09 vs target weights 33.33%


# 问题
1. 如果奖励不够稀疏，它还不如随机采样，随机采样的时间复杂度基本上是O（1），而他的采样复杂度和更新复杂度都比随机高
2. 消耗大量CPU资源
3. 抽样不均匀可能会导致过拟合